# Creating Binomial Tree

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

## Defining the Instance Variables

When deducing the value of an option (for example) we consider a few key component inherent to the method. These are

- Strike price ($E$)
- Expiration date ($T$)
- Factor for when an increase occurs ($u$)
- Factor for when a decrease occurs ($d$)
- Option type (call or put)
- Contract type (Eurpean or American)

## Defining the Methods For Pricing

### Delta

The binomial tree method incorporates a delta hedged portfolio

\begin{equation}
\Pi = V - \Delta S
\end{equation}

where

- $\Pi$ is the value of the portfolio ($)
- $V$ is the price of the option ($)
- $\Delta$ is the propotion of stock we should short in order to hedge our portfolio
- $S$ is the stock price ($)


The deifnition of $\Delta$ is $\partial V / \partial S$ but for discrete time cases such as the binomial method $\Delta$ can take a simpler definition. Consider at a single split in the tree the value of the stock price increases to $S^{+}$ ($uS$) such that the underlying increases to $V^-$ or the stock price decreases to $S^{-}$ ($dS$) such that the underlying decreases to $V^-$. Then

\begin{equation}
\Delta = \frac{\partial V}{\partial S} = \frac{V^+ - V^-}{S^+ - S^-} = \frac{V^+ - V^-}{(u-d)S}
\end{equation}

(More information on how this equation is obtained and on the method as a whole is available at
[Investopedia](https://www.investopedia.com/terms/b/binomialoptionpricing.asp).
Alternatively, see books such as *Paul Wilmott Introduces Quantitative Finance* or
*Options, Futures and Other Derivatives* by John C. Hull are great resources.)

In [ ]:
def calc_delta(V_p, V_m, u, d, S):
    delta = (V_p - V_m) / (u*S - d*S)
    return delta

### Pricing the current value of the option

This is where the principle of no arbitrage is applied. The current value the option should be equal to the discounted value of the option at expiraiton (otherwise there is an arbitrage oppotunity). Therefore we can implement the following equation:

\begin{equation}
V - \Delta S = (V^+ - \Delta S^{+})e^{-rt}
\end{equation}

where $r$ is the discount-free rate. Rearranging for V we get

\begin{equation}
V = \Delta S (1-ue^{-rt}) + V^{+}e^{-rt}.
\end{equation}

Then subbing in our definition for $S^+$ and $\Delta$,

\begin{equation}
V = \frac{V^{+}(1 - de^{rt}) + V^{-}(ue^{-rt} - 1)}{u - d}.
\end{equation}

Which when defining p as 

\begin{equation}
p = \frac{e^{rt} - d}{u - d}
\end{equation}

We obtain

\begin{equation}
V = e^{-rt}\left[pV^{+} + (1-p)V^{-}\right].
\end{equation}



In [5]:
def option_val(r, u, d, V_p, V_m, t):
    p = (np.exp ** (r*t) - d) / (u-d)
    V = np.exp **(-r*t) * (p * V_p + (1-p)* V_m)
    return V

#### European or American?

Each node on our tree sbould finalse a value for the option at the node for whether it is an Amercian or European option. With American options we have the advantage of early excerise. For an American option, we need to detwrmine whether $max(V-E,0)$ is greater than the value calcualted from using the no-arbitrage method. If so, the value of the option is updated as $max(V-E,0)$.

## Creating the Tree

A binomial tree is made up of nodes that have a maximum of two leaves or branches. Each of these branches is the node of another binary tree itself. The binary tree is a classic example of a data structure built on iteration. However, each node has to have examctly two leaves, so the process can be created with an array $S^{j}_{i}$ where $i$ is the depth of the tree and $j$ is the number of up moves. This means that $i-j$ is the number of down moves. Therefore we have

\begin{equation}
S^{j}_{i} = S_{0}u^{j}d^{i-j}
\end{equation}

In [8]:
def stock_price(S_0, u, d, steps):
    S = []
    for i in range(steps):
        depth = []
        for j in range(i+1):
            depth.append(round(S_0 * (u**j ) * (d**(i-j)), 4))
        S.append(depth)
    return S

In [11]:
S = stock_price(100, 1.1, 1/1.1, 4)
print(S)

[[100.0], [90.9091, 110.0], [82.6446, 100.0, 121.0], [75.1315, 90.9091, 110.0, 133.1]]


The option price $V$ is determined by going back through the tree. The last branches can be calcualted using the formula for the option and expiry:

\begin{equation}
max(S-E,0)
\end{equation}

for a call. For a put

\begin{equation}
max(E-S,0)
\end{equation}



In [19]:
def option_exp(S, E, typ):
    if typ == 'call':
        return max(S-E,0)
    elif typ == 'put':
        return max(E-S,0)
    else:
        raise ValueError("typ must be either call 'call' or 'put'")

In [20]:
option_exp(100, 90, 'call')

10

In [24]:
def backprop(S_prices, E, u, d, typ):
    V_prices = []
    n = len(S_prices)
    for i in range(1, n+1):
        V_prices.append([0]*i)
    
    exp_V = V_prices[-1]
    exp_S = S_prices[-1]
    for i in range(len(exp_V)):
        exp_V[i] = round(option_exp(exp_S[i], E, typ),4)

    return V_prices


   # for i in range(n-1, -1, -1): # Reverse from index n-1 to 0

In [27]:
print(S[-1])
print(backprop(S, 90, 1.1, 1/1.1, 'call')[-1])

[75.1315, 90.9091, 110.0, 133.1]
[0, 0.9091, 20.0, 43.1]
